In [1]:
import requests
import json
import pandas as pd
import pandas_gbq
from pandas.io.json import json_normalize
from pandas.io import gbq
from google.cloud import bigquery
from google.oauth2 import service_account

# variables
url = 'https://nanit-bi-assginment.s3.amazonaws.com/shippingdata.json'

# read json data to python
data = json.loads(requests.get(url).text)

In [2]:
# dim_orders table
dim_orders_dataframe = pd.DataFrame(data['Order'], columns=['OrderId', 'OrderDate', 'OrderSource','CurrencyCode'])
dim_orders_dataframe.to_gbq(destination_table='nanit12345.dim_orders', project_id='nanit-sql-db', if_exists='replace')


1it [00:05,  5.52s/it]


In [3]:
# fact_orders table
fact_orders_dataframe = pd.DataFrame(data['Order'], columns=['OrderId', 'Total', 'TotalTax'])
fact_orders_dataframe.to_gbq(destination_table='nanit12345.fact_orders', project_id='nanit-sql-db', if_exists='replace')

1it [00:08,  8.57s/it]


In [6]:
# fact_order_lines table
flat_order_lines=[]
for order in data['Order']:
    for order_line in order['OrderLines'] or []:
        result = order['OrderId'], order_line['ProductCode'], order_line['UnitCost'], order_line['Quantity']
        flat_order_lines.append(result)
        
flat_order_lines_dataframe = pd.DataFrame(flat_order_lines, columns=['OrderId', 'ProductCode', 'UnitCost','Quantity'])
flat_order_lines_dataframe.to_gbq(destination_table='nanit12345.fact_order_lines', project_id='nanit-sql-db', if_exists='replace')

1it [00:04,  4.68s/it]


In [17]:
# dim_products table
flat_products=[]
for order in data['Order']:
    for order_line in order['OrderLines'] or []:
        result = order_line['ProductCode'], order_line['ProductDescription']
        flat_products.append(result)

dim_products_dataframe = pd.DataFrame(flat_products, columns=['ProductCode','ProductDescription'])        
distinct_dim_products_dataframe = dim_products_dataframe.drop_duplicates()
distinct_dim_products_dataframe.to_gbq(destination_table='nanit12345.dim_products', project_id='nanit-sql-db', if_exists='replace')


1it [00:09,  9.06s/it]


In [8]:
# dim_dispatches
flat_dispatches=[]
for order in data['Order']:
    for dispatch in order['Dispatches'] or []:
        result = dispatch['DispatchReference'], dispatch['DispatchDate'], dispatch['Carrier'], dispatch['TrackingNumber'], dispatch['TrackingURL']
        flat_dispatches.append(result)
        
dim_dispatches_dataframe = pd.DataFrame(flat_dispatches, columns=['DispatchReference','DispatchDate','Carrier','TrackingNumber','TrackingURL'])
dim_dispatches_dataframe.to_gbq(destination_table='nanit12345.dim_dispatches', project_id='nanit-sql-db', if_exists='replace')

1it [00:07,  7.08s/it]


In [9]:
# fact_dispatch_lines table
flat_dispatch_lines=[]
for order in data['Order']:
    for dispatch in order['Dispatches'] or []:
        for dispatch_line in dispatch['DispatchedLines'] or []:
            result = order['OrderId'], dispatch['DispatchedLines'], dispatch_line['ProductCode'], dispatch_line['Quantity']
            flat_dispatch_lines.append(result)
        
fact_dispatch_lines_dataframe = pd.DataFrame(flat_dispatch_lines, columns=['OrderId','DispatchReference','ProductCode','Quantity'])
fact_dispatch_lines_dataframe.to_gbq(destination_table='nanit12345.fact_dispatch_lines', project_id='nanit-sql-db', if_exists='replace')


1it [00:09,  9.05s/it]


In [19]:
# create dwh_order_lines_details_etl table, the base for the BI dashboard 
query = """
select 
dispatch_lines.OrderId,
dispatch_lines.Quantity,
dispatch_lines.ProductCode,

dispatches.Carrier,
dispatches.DispatchDate,

orders.OrderSource,

products.ProductDescription

from nanit12345.fact_dispatch_lines as dispatch_lines
    left join nanit12345.dim_dispatches as dispatches
        on dispatch_lines.DispatchReference = dispatches.DispatchReference
        
  left join nanit12345.dim_orders as orders
        on dispatch_lines.OrderId = orders.OrderId
        
  left join (select * from nanit12345.dim_products where ProductDescription<>'Non Camera Hardware (Bundle)') as products
        on dispatch_lines.ProductCode = products.ProductCode
"""
query_result_dataframe = pandas_gbq.read_gbq(query, project_id='nanit-sql-db' )
query_result_dataframe.to_gbq(destination_table='nanit12345.dwh_order_lines_details_etl', project_id='nanit-sql-db', if_exists='replace')

Downloading: 100%|████████████████████████| 465/465 [00:00<00:00, 487.43rows/s]
1it [00:04,  4.47s/it]
